## Reproduce results from “Modeling BBR’s Interactions with Loss-Based Congestion Control”

This notebook will reproduce selected findings from

> Ranysha Ware, Matthew K. Mukerjee, Srinivasan Seshan, and Justine Sherry. 2019. Modeling BBR’s Interactions with Loss-Based Congestion Control. In Proceedings of the Internet Measurement Conference (IMC ’19). Association for Computing Machinery, New York, NY, USA, 137–143. https://doi.org/10.1145/3355369.3355604

specifically,

-   Figure 1b and 1c (initial measurements of BBR’s empirical behaviors)
-   Figure 2: BBR vs Cubic in a 40ms × 10Mbps network (convergence time and goodput for 1 BBR and 1 Cubic flow over varying queue sizes)
-   Figure 3: BBR and Cubic or Reno’s queue when competing for 4 minutes over a network with a 64 BDP (1024 packet) queue

The following group of cells installs prerequisites for data analysis and visualization:

In [ ]:
!pip install seaborn

In [ ]:
import time
import pandas as pd
from io import StringIO
import seaborn as sns 
import matplotlib.pyplot as plt

### Configure environment

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
conf = fablib.show_config()

### Define configuration for this experiment

In [ ]:
slice_name="ware-" + fablib.get_bastion_username()

node_conf = [
 {'name': "sender",    'cores': 2, 'ram': 4, 'disk': 10, 'image': 'default_ubuntu_22', 'packages': ['iperf3']}, 
 {'name': "receiver",  'cores': 2, 'ram': 4, 'disk': 10, 'image': 'default_ubuntu_22', 'packages': ['iperf3']}, 
 {'name': "router",    'cores': 2, 'ram': 4, 'disk': 10, 'image': 'default_ubuntu_22', 'packages': []}
]
net_conf = [
 {"name": "net0", "subnet": "10.0.0.0/24", "nodes": [{"name": "sender",   "addr": "10.0.0.100"}, {"name": "router", "addr": "10.0.0.1"}]},
 {"name": "net1", "subnet": "10.0.1.0/24", "nodes": [{"name": "receiver", "addr": "10.0.1.100"}, {"name": "router", "addr": "10.0.1.1"}]},
]
route_conf = [
 {"addr": "10.0.1.0/24", "gw": "10.0.0.1", "nodes": ["sender"]}, 
 {"addr": "10.0.0.0/24", "gw": "10.0.1.1", "nodes": ["receiver"]}
]
exp_conf = {'cores': sum([ n['cores'] for n in node_conf]), 'nic': sum([len(n['nodes']) for n in net_conf]) }

### Reserve resources

Now, we are ready to reserve resources!

First, make sure you don’t already have a slice with this name:

In [ ]:
try:
    slice = fablib.get_slice(slice_name)
    print("You already have a slice by this name!")
    print("If you previously reserved resources, skip to the 'log in to resources' section.")
except:
    print("You don't have a slice named %s yet." % slice_name)
    print("Continue to the next step to make one.")
    slice = fablib.new_slice(name=slice_name)

We will select a random site that has sufficient resources for our experiment:

In [ ]:
while True:
    site_name = fablib.get_random_site()
    if ( (fablib.resources.get_core_available(site_name) > 1.2*exp_conf['cores']) and
        (fablib.resources.get_component_available(site_name, 'SharedNIC-ConnectX-6') > 1.2**exp_conf['nic']) ):
        break

fablib.show_site(site_name)

Then we will add hosts and network segments:

In [ ]:
# this cell sets up the nodes
for n in node_conf:
    slice.add_node(name=n['name'], site=site_name, 
                   cores=n['cores'], 
                   ram=n['ram'], 
                   disk=n['disk'], 
                   image=n['image'])

In [ ]:
# this cell sets up the network segments
for n in net_conf:
    ifaces = [slice.get_node(node["name"]).add_component(model="NIC_Basic", 
                                                 name=n["name"]).get_interfaces()[0] for node in n['nodes'] ]
    slice.add_l2network(name=n["name"], type='L2Bridge', interfaces=ifaces)

The following cell submits our request to the FABRIC site. The output of this cell will update automatically as the status of our request changes.

-   While it is being prepared, the “State” of the slice will appear as “Configuring”.
-   When it is ready, the “State” of the slice will change to “StableOK”.

You may prefer to walk away and come back in a few minutes (for simple slices) or a few tens of minutes (for more complicated slices with many resources).

In [ ]:
slice.submit()

In [ ]:
slice.get_state()
slice.wait_ssh(progress=True)

### Extend your slice

If you don’t plan to finish an experiment in one day, you can extend your slice. The following cell extends your reservation for 7 days.

In [ ]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

# Set end date to 7 days from now
end_date = (datetime.now(timezone.utc) + timedelta(days=7)).strftime("%Y-%m-%d %H:%M:%S %z")
slice.renew(end_date)

### Configure resources

Next, we will configure the resources so they are ready to use.

In [ ]:
slice = fablib.get_slice(name=slice_name)

In [ ]:
# install packages
# this will take a while and will run in background while you do other steps
for n in node_conf:
    if len(n['packages']):
        node = slice.get_node(n['name'])
        pkg = " ".join(n['packages'])
        node.execute_thread("sudo apt update; sudo DEBIAN_FRONTEND=noninteractive apt -y install %s" % pkg)

In [ ]:
# bring interfaces up and either assign an address (if there is one) or flush address
from ipaddress import ip_address, IPv4Address, IPv4Network

for net in net_conf:
    for n in net['nodes']:
        if_name = n['name'] + '-' + net['name'] + '-p1'
        iface = slice.get_interface(if_name)
        iface.ip_link_up()
        if n['addr']:
            iface.ip_addr_add(addr=n['addr'], subnet=IPv4Network(net['subnet']))
        else:
            iface.get_node().execute("sudo ip addr flush dev %s"  % iface.get_device_name())

In [ ]:
# make sure all interfaces are brought up
for iface in slice.get_interfaces():
    iface.ip_link_up()

In [ ]:
# prepare a "hosts" file that has names and addresses of every node
hosts_txt = [ "%s\t%s" % ( n['addr'], n['name'] ) for net in net_conf  for n in net['nodes'] if type(n) is dict and n['addr']]
for n in slice.get_nodes():
    for h in hosts_txt:
        n.execute("echo %s | sudo tee -a /etc/hosts" % h)

In [ ]:
# enable IPv4 forwarding on all nodes
for n in slice.get_nodes():
    n.execute("sudo sysctl -w net.ipv4.ip_forward=1")

In [ ]:
# set up static routes
for rt in route_conf:
    for n in rt['nodes']:
        slice.get_node(name=n).ip_route_add(subnet=IPv4Network(rt['addr']), gateway=rt['gw'])

In [ ]:
# turn off segmentation offload on interfaces
for iface in slice.get_interfaces():
    iface_name = iface.get_device_name()
    n = iface.get_node()
    offloads = ["gro", "lro", "gso", "tso"]
    for offload in offloads:
        n.execute("sudo ethtool -K %s %s off" % (iface_name, offload))

### Validate base network

Before we run any experiment, we should check the “base” network - before adding any emulated delay or rate limiting - and make sure that it will not be a limiting factor in the experiment.

In [ ]:
# check base delay
_ = slice.get_node("sender").execute("ping -c 5 receiver")

In [ ]:
# check base capacity (by sending 10 parallel flows, look at their sum throughput)
import time
_ = slice.get_node("receiver").execute("iperf3 -s -1 -D")
time.sleep(5)
_ = slice.get_node("sender").execute("iperf3 -t 30 -i 10 -P 10 -c receiver")

In [ ]:
# also check Linux kernel version on sender
_ = slice.get_node("sender").execute("uname -a")

### Draw the network topology

The following cell will draw the network topology, for your reference. The interface name and addresses of each experiment interface will be shown on the drawing.

In [ ]:
l2_nets = [(n.get_name(), {'color': 'lavender'}) for n in slice.get_l2networks() ]
l3_nets = [(n.get_name(), {'color': 'pink'}) for n in slice.get_l3networks() ]
hosts   =   [(n.get_name(), {'color': 'lightblue'}) for n in slice.get_nodes()]
nodes = l2_nets + l3_nets + hosts
ifaces = [iface.toDict() for iface in slice.get_interfaces()]
edges = [(iface['network'], iface['node'], 
          {'label': iface['physical_dev'] + '\n' + iface['ip_addr'] + '\n' + iface['mac']}) for iface in ifaces]

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
plt.figure(figsize=(len(nodes),len(nodes)))
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)
pos = nx.spring_layout(G)
nx.draw(G, pos, node_shape='s',  
        node_color=[n[1]['color'] for n in nodes], 
        node_size=[len(n[0])*400 for n in nodes],  
        with_labels=True);
nx.draw_networkx_edge_labels(G,pos,
                             edge_labels=nx.get_edge_attributes(G,'label'),
                             font_color='gray',  font_size=8, rotate=False);

### Log into resources

Now, we are finally ready to log in to our resources over SSH! Run the following cells, and observe the table output - you will see an SSH command for each of the resources in your topology.

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
slice_info = [{'Name': n.get_name(), 'SSH command': n.get_ssh_command()} for n in slice.get_nodes()]
pd.DataFrame(slice_info).set_index('Name')

Now, you can open an SSH session on any of the resources as follows:

-   in Jupyter, from the menu bar, use File \> New \> Terminal to open a new terminal.
-   copy an SSH command from the table, and paste it into the terminal. (Note that each SSH command is a single line, even if the display wraps the text to a second line! When you copy and paste it, paste it all together.)

You can repeat this process (open several terminals) to start a session on each resource. Each terminal session will have a tab in the Jupyter environment, so that you can easily switch between them.

### Configure the network capacity and delay

In this section, we configure the bottleneck link to have a 40Mbps capacity and 10ms delay. We will initialize the queue size to 32 BDP (although we will change this later).

In [ ]:
exp = {'rtt': 40, 'bandwidth': 10 }
bdp_kbyte = exp['rtt']*exp['bandwidth']/8

router_node = slice.get_node("router")
router_ingress_iface = router_node.get_interface(network_name = "net0")
router_ingress_name = router_ingress_iface.get_device_name()
router_egress_iface  = router_node.get_interface(network_name = "net1")
router_egress_name = router_egress_iface.get_device_name()

router_node.execute("sudo tc qdisc del dev " + router_ingress_name + " root")
router_node.execute("sudo tc qdisc del dev " + router_egress_name + " root")

# set up RTT
router_node.execute("sudo tc qdisc replace dev " + router_ingress_name + " root netem delay " + str(exp['rtt']) + "ms limit 10000")
# set up rate limit, buffer limit
router_node.execute("sudo tc qdisc replace dev " + router_egress_name + " root handle 1: htb default 3")
router_node.execute("sudo tc class add dev " + router_egress_name + " parent 1: classid 1:3 htb rate " + str(exp['bandwidth']) + "Mbit")
router_node.execute("sudo tc qdisc add dev " + router_egress_name + " parent 1:3 bfifo limit " + str(bdp_kbyte*32) + "kb")


Then, we validate the new network setting.

In [ ]:
# check base delay
_ = slice.get_node("sender").execute("ping -c 5 receiver")

In [ ]:
# check base capacity (by sending 10 parallel flows, look at their sum throughput)
import time
_ = slice.get_node("receiver").execute("iperf3 -s -1 -D")
time.sleep(5)
_ = slice.get_node("sender").execute("iperf3 -t 30 -i 10 -P 10 -c receiver")

### Reproduce Figure 1b

In [ ]:
router_node = slice.get_node("router")
sender_node = slice.get_node("sender")
receiver_node = slice.get_node("receiver")

In [ ]:
import itertools
exp_factors = { 
    'bufcap': [0.25, 0.5] + [2**n for n in range(8)],
    #'bufcap': [0.25],
    'duration': [240],
    'loss_cc': ['cubic', 'reno'],
    #'loss_cc': ['cubic'],
    'trial': [1]
}
factor_names = [k for k in exp_factors]
factor_lists = list(itertools.product(*exp_factors.values()))
exp_lists = [dict(zip(factor_names, factor_l)) for factor_l in factor_lists]

In [ ]:
exp_lists

In [ ]:
import time

# make sure BBR is available
sender_node.execute("sudo modprobe tcp_bbr")
receiver_node.execute("sudo modprobe tcp_bbr")

for exp in exp_lists:
    print("Running:",exp)
    sender_node.execute("sudo modprobe tcp_bbr")
    # set router buffer limit 
    router_node.execute("sudo tc qdisc replace dev " + router_egress_name + " parent 1:3 bfifo limit " + str(bdp_kbyte*exp['bufcap']) + "kb")

    # clean up
    receiver_node.execute("sudo killall iperf3")
    sender_node.execute("sudo killall iperf3")
    
    time.sleep(5) 
    
    # start an iperf3 receiver for the BBR flow
    receiver_node.execute_thread("iperf3 -s -1 -i 1 -fm --logfile fig1b_bbr_" + str(exp['bufcap']) + "_bbrV" + exp['loss_cc'] + ".txt")
    #receiver_node.execute("iperf3 -s -D -1 -i 1 -fm --logfile fig1b_bbr_" + str(exp['bufcap']) + "_bbrV" + exp['loss_cc'] + ".txt")
    # start an iperf3 receiver for the loss based CC flows (Cubic & Reno)
    receiver_node.execute_thread("iperf3 -s -1 -i 1 -fm --logfile fig1b_" + exp['loss_cc'] + "_" + str(exp['bufcap']) + "_bbrV" + exp['loss_cc'] + ".txt -p 5301")
    #receiver_node.execute("iperf3 -s -D -1 -i 1 -fm --logfile fig1b_" + exp['loss_cc'] + "_" + str(exp['bufcap']) + "_bbrV" + exp['loss_cc'] + ".txt -p 5301")

    time.sleep(10) 

    # start an iperf3 sender for the BBR flow
    sender_node.execute_thread("iperf3 -c receiver -fm -t " + str(exp['duration']) + " -C bbr ")
    # start an iperf3 sender for the loss based CC flows (Cubic & Reno)
    sender_node.execute_thread("iperf3 -c receiver -fm -t " + str(exp['duration']) + " -C " + exp['loss_cc'] + " -p 5301")

    time.sleep(exp['duration'] + 10)
    
print("done")

In [ ]:
df = pd.DataFrame(columns=['bufcap', 'combo', 'cc', 'goodput'])
for exp in exp_lists:

    bbr_file = "fig1b_bbr_" + str(exp['bufcap']) + "_bbrV" + exp['loss_cc'] + ".txt"
    tput_bbr = receiver_node.execute("cat /home/ubuntu/figure1b/" + bbr_file + " | grep 'receiver' | awk -F '-' '{print $2}' | awk '{print $5}'", quiet=True)
    df_dict = {'bufcap': exp['bufcap'], 'combo': "BBR-" + exp['loss_cc'], 'cc': 'BBR', 'goodput': float(tput_bbr[0].strip())}
    df = pd.concat([df, pd.DataFrame(df_dict, index=[0])], ignore_index=True)

    loss_file = "fig1b_" + exp['loss_cc'] + "_" + str(exp['bufcap']) + "_bbrV" + exp['loss_cc'] + ".txt"
    tput_loss = receiver_node.execute("cat /home/ubuntu/figure1b/" + loss_file + " | grep 'receiver' | awk -F '-' '{print $2}' | awk '{print $5}'", quiet=True)
    df_dict = {'bufcap': exp['bufcap'], 'combo': "BBR-" + exp['loss_cc'], 'cc': exp['loss_cc'], 'goodput': float(tput_loss[0].strip())}
    df = pd.concat([df, pd.DataFrame(df_dict, index=[0])], ignore_index=True)

In [ ]:
df['buf_str'] = df['bufcap'].astype(str)
g = sns.FacetGrid(df, row="combo", legend_out=True, aspect=2);
g = g.map(sns.lineplot, "buf_str", "goodput",  hue=df.cc, style=df.cc, markers="o", markersize=6);
g.set_axis_labels("Buffer size (BDP)", "Goodput (Mbps)");
g.set(ylim=(0, 10))
g.axes.flat[0].grid(True)
g.axes.flat[1].grid(True)
plt.legend();
plt.savefig("figure1b.pdf")

### Reproduce Figure 1c

In [ ]:
router_node = slice.get_node("router")
sender_node = slice.get_node("sender")
receiver_node = slice.get_node("receiver")

In [ ]:
# set router buffer limit to 32 BDP
router_node.execute("sudo tc qdisc replace dev " + router_egress_name + " parent 1:3 bfifo limit " + str(bdp_kbyte*32) + "kb")

# make sure BBR is available
sender_node.execute("sudo modprobe tcp_bbr")

# clean up
receiver_node.execute("sudo killall iperf3")
receiver_node.execute("rm fig1c_bbr.txt")
receiver_node.execute("rm fig1c_cubic.txt")

# start an iperf3 receiver for the BBR flow
receiver_node.execute_thread("iperf3 -s -1 -i 1 -fm --logfile fig1c_bbr.txt")
# start an iperf3 receiver for the Cubic flows
receiver_node.execute_thread("iperf3 -s -1 -i 1 -fm --logfile fig1c_cubic.txt -p 5301")

time.sleep(5) 

# start an iperf3 sender for the BBR flow
sender_node.execute_thread("iperf3 -c receiver -fm -t 300 -C bbr ")
# start an iperf3 receiver for the Cubic flows
sender_node.execute_thread("iperf3 -c receiver -fm -t 300 -C cubic -P 16 -p 5301")

time.sleep(305)

In [ ]:
tput_bbr = receiver_node.execute("head --lines=-5 /home/ubuntu/figure1c/fig1c_bbr.txt | grep 'Mbits/sec' | awk -F '-' '{print $2}' | awk '{print $1\",\"$5}'", quiet=True)
df_bbr = pd.read_csv(StringIO(tput_bbr[0]), names = ['time','goodput'])

tput_cubic = receiver_node.execute("head --lines=-37 /home/ubuntu/figure1c/fig1c_cubic.txt | grep 'SUM' | awk -F '-' '{print $2}' | awk '{print $1\",\"$5}'", quiet=True)
df_cubic = pd.read_csv(StringIO(tput_cubic[0]), names = ['time','goodput'])

_ = plt.figure(figsize=(12,6))
_ = plt.plot(df_bbr.time, df_bbr.goodput, label="1 BBR flow")
_ = plt.plot(df_cubic.time, df_cubic.goodput, label="Sum of 16 CUBIC flows")
_ = plt.legend(loc="upper center", ncol=2)
_ = plt.ylabel("Goodput (Mbps)")
_ = plt.xlabel("Time (s)")
plt.savefig("figure1c.pdf")

### Reproduce Figure 2a

In [ ]:
router_node = slice.get_node("router")
sender_node = slice.get_node("sender")
receiver_node = slice.get_node("receiver")

In [ ]:
import itertools
exp_factors = { 
    'bufcap': [2**n for n in range(7)],
    'duration': [660],
    'loss_cc': ['cubic'],
    'trial': [1]
}
factor_names = [k for k in exp_factors]
factor_lists = list(itertools.product(*exp_factors.values()))
exp_lists = [dict(zip(factor_names, factor_l)) for factor_l in factor_lists]

In [ ]:
exp_lists

In [ ]:
import time

# make sure BBR is available
sender_node.execute("sudo modprobe tcp_bbr")
receiver_node.execute("sudo modprobe tcp_bbr")

for exp in exp_lists:
    print("Running:",exp)
    sender_node.execute("sudo modprobe tcp_bbr")
    # set router buffer limit 
    router_node.execute("sudo tc qdisc replace dev " + router_egress_name + " parent 1:3 bfifo limit " + str(bdp_kbyte*exp['bufcap']) + "kb")

    # clean up
    receiver_node.execute("sudo killall iperf3")
    sender_node.execute("sudo killall iperf3")
    
    time.sleep(5) 
    
    # start an iperf3 receiver for the BBR flow
    receiver_node.execute_thread("iperf3 -s -1 -i 1 -fm --logfile fig2a_bbr_" + str(exp['bufcap']) + "_bbrV" + exp['loss_cc'] + ".txt")
    # start an iperf3 receiver for the loss based CC flows (Cubic & Reno)
    receiver_node.execute_thread("iperf3 -s -1 -i 1 -fm --logfile fig2a_" + exp['loss_cc'] + "_" + str(exp['bufcap']) + "_bbrV" + exp['loss_cc'] + ".txt -p 5301")

    time.sleep(10) 

    # start an iperf3 sender for the BBR flow
    sender_node.execute_thread("iperf3 -c receiver -fm -t " + str(exp['duration']) + " -C bbr ")
    # start an iperf3 sender for the loss based CC flows (Cubic & Reno)
    sender_node.execute_thread("iperf3 -c receiver -fm -t " + str(exp['duration']) + " -C " + exp['loss_cc'] + " -p 5301")

    time.sleep(exp['duration'] + 10)
    
print("done")

In [ ]:
experiment_dfs = {}

for exp in exp_lists:

    bbr_file = "fig2a_bbr_" + str(exp['bufcap']) + "_bbrV" + exp['loss_cc'] + ".txt"
    tput_bbr =  receiver_node.execute("head --lines=-5 /home/ubuntu/figure2a/"+bbr_file+" | grep 'Mbits/sec' | awk -F '-' '{print $2}' | awk '{print $1\",\"$5}'", quiet=True)
    df_bbr = pd.read_csv(StringIO(tput_bbr[0]), names = ['time','goodput'])
    experiment_key = f"bbr_{exp['bufcap']}_{exp['loss_cc']}"
    experiment_dfs[experiment_key] = df_bbr


    loss_file = "fig2a_" + exp['loss_cc'] + "_" + str(exp['bufcap']) + "_bbrV" + exp['loss_cc'] + ".txt"
    tput_loss = receiver_node.execute("head --lines=-5 /home/ubuntu/figure2a/"+loss_file+" | grep 'Mbits/sec' | awk -F '-' '{print $2}' | awk '{print $1\",\"$5}'", quiet=True)
    df_loss = pd.read_csv(StringIO(tput_loss[0]), names = ['time','goodput'])
    experiment_key = f"loss_{exp['bufcap']}_{exp['loss_cc']}"
    experiment_dfs[experiment_key] = df_loss

In [ ]:
for key, df in experiment_dfs.items():
    # Construct file name based on the experiment key
    file_name = f"{key}.csv"
    # Save DataFrame to CSV
    df.to_csv(file_name, index=False)
    print(f"Saved {file_name}")

In [ ]:
def find_convergence(df, percentage_change, time_period):
    """
    Find the first time instance when the throughput (goodput) does not change
    by more than X% for T seconds.

    :param df: DataFrame with 'time' and 'goodput' columns
    :param percentage_change: The maximum allowed relative change in goodput (as a percentage)
    :param time_period: The period of time for which we require stability (in seconds)
    :return: The time of convergence, or None if convergence was not found
    """
    # Convert percentage to a proportion
    max_change = percentage_change / 100.0
    
    # Initialize variables
    start_time = df['time'].iloc[0]
    previous_goodput = df['goodput'].iloc[0]
    
    # Iterate over the DataFrame to check for periods of stability
    for i in range(1, len(df)):
        current_time = df['time'].iloc[i]
        current_goodput = df['goodput'].iloc[i]
        
        # Calculate the relative change in goodput
        if previous_goodput != 0:  # To avoid division by zero
            change = abs((current_goodput - previous_goodput) / previous_goodput)
        else:
            change = 0
        
        # If the change is more than the allowed max change, reset the start time
        if change > max_change:
            start_time = current_time
            previous_goodput = current_goodput
        elif (current_time - start_time) >= time_period:
            # If the period of stability is greater than or equal to T seconds, return the start time
            return start_time
    
    # If the loop completes without returning, convergence was not found
    return None

In [ ]:
def find_convergence(df, percentage_change, time_period):
    """
    Find the first time instance when the average throughput (goodput) does not change
    by more than X% for at least T seconds.

    :param df: DataFrame with 'time' and 'goodput' columns
    :param percentage_change: The maximum allowed relative change in goodput (as a percentage)
    :param time_period: The period of time for which we require stability (in seconds)
    :return: The time of convergence, or None if convergence was not found
    """
    # Convert percentage to a proportion
    max_change = percentage_change / 100.0

    # Get the initial rolling average
    rolling_avg = df['goodput'].rolling(window=time_period, min_periods=1).mean()

    for i in range(time_period, len(df)):
        # Calculate the percentage change between the current rolling average and the previous period's rolling average
        if rolling_avg[i - time_period] != 0:  # To avoid division by zero
            change = abs((rolling_avg[i] - rolling_avg[i - time_period]) / rolling_avg[i - time_period])
        else:
            change = 0

        # Check if the percentage change is within the threshold
        if change <= max_change:
            # Return the start time of the window where convergence criteria is met
            return df['time'].iloc[i - time_period + 1]

    # If the loop completes without returning, convergence was not found
    return None

In [ ]:
X = 0.1
T = 120

#X = 2
#T = 20


convergence_times = {}

for key, df in experiment_dfs.items():
    # Check if the key starts with 'bbr_'
    if key.startswith('bbr_'):
        
        # Call the find_convergence function with the DataFrame
        convergence_time = find_convergence(df, X, T)
        
        # Store the convergence time in the results dictionary
        convergence_times[key] = convergence_time

In [ ]:
convergence_times

In [ ]:
import numpy as np

# Given queue sizes
queue_sizes = [2**n for n in range(7)]

# Extract the convergence times into a list, in the same order as queue_sizes
times = [convergence_times.get(f'bbr_{q}_cubic', 0) for q in queue_sizes]

plt.figure(figsize=(10, 5))
plt.bar(np.log2(queue_sizes), times, color=(0.1, 0.3, 0.9), width=0.6)  # Adjust width for a more compact look
plt.xlabel('Queue Size (BDP)', fontsize=14, labelpad=10)
plt.ylabel('Convergence Time (s)', fontsize=14, labelpad=10)
plt.title(f'Convergence Time vs Queue Size - {X}% Max Change over {T} Seconds Window ', fontsize=16, pad=20)
plt.xticks(np.log2(queue_sizes), [str(q) for q in queue_sizes], fontsize=12)  # Ensuring proper tick label formatting
plt.yticks(fontsize=12)
plt.grid(True, linestyle='--', linewidth=0.5)  # Adjust grid style
plt.tight_layout()  # Adjust layout to make sure everything fits without overlap
plt.savefig("figure2a.pdf")
plt.show()

### Reproduce Figure 2b

In [ ]:
router_node = slice.get_node("router")
sender_node = slice.get_node("sender")
receiver_node = slice.get_node("receiver")

In [ ]:
import itertools
exp_factors = { 
    'bufcap': [2**(n+2) for n in range(5)],
    'duration': [240, 600],
    'loss_cc': ['cubic'],
    'trial': [1]
}
factor_names = [k for k in exp_factors]
factor_lists = list(itertools.product(*exp_factors.values()))
exp_lists = [dict(zip(factor_names, factor_l)) for factor_l in factor_lists]

In [ ]:
exp_lists

In [ ]:
import time

# make sure BBR is available
sender_node.execute("sudo modprobe tcp_bbr")
receiver_node.execute("sudo modprobe tcp_bbr")

for exp in exp_lists:
    print("Running:",exp)
    sender_node.execute("sudo modprobe tcp_bbr")
    # set router buffer limit 
    router_node.execute("sudo tc qdisc replace dev " + router_egress_name + " parent 1:3 bfifo limit " + str(bdp_kbyte*exp['bufcap']) + "kb")

    # clean up
    receiver_node.execute("sudo killall iperf3")
    sender_node.execute("sudo killall iperf3")
    
    time.sleep(5) 
    
    # start an iperf3 receiver for the BBR flow
    receiver_node.execute_thread("iperf3 -s -1 -i 1 -fm --logfile fig2b_bbr_" + str(exp['bufcap']) + "_bbrV" + exp['loss_cc']+"_"+ str(exp['duration'])+ ".txt")
    #receiver_node.execute("iperf3 -s -D -1 -i 1 -fm --logfile fig1b_bbr_" + str(exp['bufcap']) + "_bbrV" + exp['loss_cc'] + ".txt")
    # start an iperf3 receiver for the loss based CC flows (Cubic & Reno)
    receiver_node.execute_thread("iperf3 -s -1 -i 1 -fm --logfile fig2b_" + exp['loss_cc'] + "_" + str(exp['bufcap']) + "_bbrV" + exp['loss_cc'] + "_"+ str(exp['duration'])+".txt -p 5301")
    #receiver_node.execute("iperf3 -s -D -1 -i 1 -fm --logfile fig1b_" + exp['loss_cc'] + "_" + str(exp['bufcap']) + "_bbrV" + exp['loss_cc'] + ".txt -p 5301")

    time.sleep(10) 

    # start an iperf3 sender for the BBR flow
    sender_node.execute_thread("iperf3 -c receiver -fm -t " + str(exp['duration']) + " -C bbr ")
    # start an iperf3 sender for the loss based CC flows (Cubic & Reno)
    sender_node.execute_thread("iperf3 -c receiver -fm -t " + str(exp['duration']) + " -C " + exp['loss_cc'] + " -p 5301")

    time.sleep(exp['duration'] + 10)
    
print("done")

In [ ]:
df = pd.DataFrame(columns=['bufcap', 'duration', 'combo', 'cc', 'goodput'])
for exp in exp_lists:

    bbr_file = "fig2b_bbr_" + str(exp['bufcap']) + "_bbrV" + exp['loss_cc']+"_"+ str(exp['duration']) + ".txt"
    tput_bbr = receiver_node.execute("cat /home/ubuntu/figure2b/" + bbr_file + " | grep 'receiver' | awk -F '-' '{print $2}' | awk '{print $5}'", quiet=True)
    df_dict = {'bufcap': exp['bufcap'], 'duration': exp['duration'], 'combo': "BBR-" + exp['loss_cc'], 'cc': 'BBR', 'goodput': float(tput_bbr[0].strip())}
    df = pd.concat([df, pd.DataFrame(df_dict, index=[0])], ignore_index=True)

    loss_file = "fig2b_" + exp['loss_cc'] + "_" + str(exp['bufcap']) + "_bbrV" + exp['loss_cc'] + "_"+ str(exp['duration']) + ".txt"
    tput_loss = receiver_node.execute("cat /home/ubuntu/figure2b/" + loss_file + " | grep 'receiver' | awk -F '-' '{print $2}' | awk '{print $5}'", quiet=True)
    df_dict = {'bufcap': exp['bufcap'],'duration': exp['duration'], 'combo': "BBR-" + exp['loss_cc'], 'cc': exp['loss_cc'], 'goodput': float(tput_loss[0].strip())}
    df = pd.concat([df, pd.DataFrame(df_dict, index=[0])], ignore_index=True)

In [ ]:
df

In [ ]:
experiment_dfs = {}
#tput_bbr =  receiver_node.execute("head --lines=-5 /home/ubuntu/figure2a/"+bbr_file+" | grep 'Mbits/sec' | awk -F '-' '{print $2}' | awk '{print $1\",\"$5}'", quiet=True)

for exp in exp_lists:
    
    if exp["duration"] ==600:
        bbr_file = "fig2b_bbr_" + str(exp['bufcap']) + "_bbrV" + exp['loss_cc']+"_"+ str(exp['duration']) + ".txt"
        tput_bbr =  receiver_node.execute("tail -n 185 /home/ubuntu/figure2b/"+bbr_file+" | head -n 180 | grep 'Mbits/sec' | awk -F '-' '{print $2}' | awk '{print $1\",\"$5}'", quiet=True)
        #print(tput_bbr)
        df_bbr = pd.read_csv(StringIO(tput_bbr[0]), names = ['time','goodput'])
        average_goodput = df_bbr['goodput'].mean()
        experiment_key = f"bbr_{exp['bufcap']}_{exp['loss_cc']}_{exp['duration']}"
        experiment_dfs[experiment_key] = average_goodput  
                                   
                                   
    #loss_file = "fig2b_" + str(exp['bufcap']) + "_bbrV" + exp['loss_cc']+"_"+ str(exp['duration']) + ".txt"
    #t_loss =  receiver_node.execute("tail -n 185 /home/ubuntu/figure2b/"+loss_file+" | head -n 180 | grep 'Mbits/sec' | awk -F '-' '{print $2}' | awk '{print $1\",\"$5}'", quiet=True)
    #df_loss = pd.read_csv(StringIO(tput_bbr[0]), names = ['time','goodput'])
    #average_goodput = df_bbr['goodput'].mean()
    #experiment_key = f"bbr_{exp['bufcap']}_{exp['loss_cc']}_{exp['duration']}"
    #experiment_dfs[experiment_key] = average_goodput

# Assuming you want to see the results:
print(experiment_dfs)

In [ ]:
bbr_df = df[df['cc'] == 'BBR'].copy()

# Convert 'duration' to a string column for labeling purposes in the plot
bbr_df['duration_str'] = bbr_df['duration'].apply(lambda x: f'{x//60} min')

# Initialize a large enough figure size for clarity
plt.figure(figsize=(10, 6))
sns.barplot(data=bbr_df, x='bufcap', y='goodput', hue='duration_str', errorbar=None)

# Add labels and title
plt.xlabel('Buffer Size (BDP)')
plt.ylabel('Goodput (Mbps)')
plt.title('BBR Throughput for Different Durations')
plt.legend(title='Duration')
plt.show()

In [ ]:
convergence_data = {
    'bufcap': [int(key.split('_')[1]) for key in experiment_dfs.keys()],
    'goodput': list(experiment_dfs.values()),
    'duration_str': ['After Convergence'] * len(experiment_dfs)
}
conv_df = pd.DataFrame(convergence_data)

# Create a DataFrame from the new data
conv_df = pd.DataFrame(convergence_data)

# Concatenate this new DataFrame with the original 'bbr_df'
bbr_df_final = pd.concat([bbr_df, conv_df], ignore_index=True)

# Initialize a large enough figure size for clarity
plt.figure(figsize=(12, 8))
sns.barplot(data=bbr_df_final, x='bufcap', y='goodput', hue='duration_str', errorbar=None)

# Add labels and title
plt.xlabel('Buffer Size (BDP)')
plt.ylabel('BBR\'s Goodput (Mbps)')
plt.title('BBR Throughput for Different Durations')
plt.legend()
plt.savefig("figure2b.pdf")
plt.show()

### Reproduce Figure 3

In [ ]:
router_node = slice.get_node("router")
sender_node = slice.get_node("sender")
receiver_node = slice.get_node("receiver")

In [ ]:
import itertools
exp_factors = { 
    'bufcap': [64],
    'duration': [240],
    'loss_cc': ['reno','cubic'],
    'trial': [1]
}
factor_names = [k for k in exp_factors]
factor_lists = list(itertools.product(*exp_factors.values()))
exp_lists = [dict(zip(factor_names, factor_l)) for factor_l in factor_lists]

In [ ]:
exp_lists

In [ ]:
import time

# make sure BBR is available
sender_node.execute("sudo modprobe tcp_bbr")
receiver_node.execute("sudo modprobe tcp_bbr")

for exp in exp_lists:
    print("Running:",exp)
    sender_node.execute("sudo modprobe tcp_bbr")
    # set router buffer limit 
    router_node.execute("sudo tc qdisc replace dev " + router_egress_name + " parent 1:3 bfifo limit " + str(bdp_kbyte*exp['bufcap']) + "kb")

    # clean up
    receiver_node.execute("sudo killall iperf3")
    sender_node.execute("sudo killall iperf3")
    
    time.sleep(5) 
    
    # start an iperf3 receiver for the BBR flow
    receiver_node.execute_thread("iperf3 -s -1 -i 1 -fm --logfile fig3_bbr_" + str(exp['bufcap']) + "_bbrV" + exp['loss_cc'] + ".txt")
    # start an iperf3 receiver for the loss based CC flows (Cubic & Reno)
    receiver_node.execute_thread("iperf3 -s -1 -i 1 -fm --logfile fig3_" + exp['loss_cc'] + "_" + str(exp['bufcap']) + "_bbrV" + exp['loss_cc'] + ".txt -p 5301")

    time.sleep(10) 

    # start an iperf3 sender for the BBR flow
    sender_node.execute_thread("iperf3 -c receiver -fm -t " + str(exp['duration']) + " -C bbr ")
    # start an iperf3 sender for the loss based CC flows (Cubic & Reno)
    sender_node.execute_thread("iperf3 -c receiver -fm -t " + str(exp['duration']) + " -C " + exp['loss_cc'] + " -p 5301")

    time.sleep(exp['duration'] + 10)
    
print("done")

In [ ]:
experiment_dfs = {}

for exp in exp_lists:

    bbr_file = "fig3_bbr_" + str(exp['bufcap']) + "_bbrV" + exp['loss_cc'] + ".txt"
    tput_bbr =  receiver_node.execute("head --lines=-5 /home/ubuntu/figure3/"+bbr_file+" | grep 'Mbits/sec' | awk -F '-' '{print $2}' | awk '{print $1\",\"$5}'", quiet=True)
    df_bbr = pd.read_csv(StringIO(tput_bbr[0]), names = ['time','goodput'])
    experiment_key = f"bbr_{exp['bufcap']}_{exp['loss_cc']}"
    experiment_dfs[experiment_key] = df_bbr


    loss_file = "fig3_" + exp['loss_cc'] + "_" + str(exp['bufcap']) + "_bbrV" + exp['loss_cc'] + ".txt"
    tput_loss = receiver_node.execute("head --lines=-5 /home/ubuntu/figure3/"+loss_file+" | grep 'Mbits/sec' | awk -F '-' '{print $2}' | awk '{print $1\",\"$5}'", quiet=True)
    df_loss = pd.read_csv(StringIO(tput_loss[0]), names = ['time','goodput'])
    experiment_key = f"loss_{exp['bufcap']}_{exp['loss_cc']}"
    experiment_dfs[experiment_key] = df_loss

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))  


for exp in exp_lists:
    bbr_key = f"bbr_{exp['bufcap']}_{exp['loss_cc']}"
    loss_key = f"loss_{exp['bufcap']}_{exp['loss_cc']}"
    
    buffer_size=(bdp_kbyte*exp['bufcap']*1000)/1500
    
    
    
    bbr_queue_occupancy = (experiment_dfs[bbr_key].goodput * 0.1)*buffer_size
    loss_queue_occupancy = (experiment_dfs[loss_key].goodput * 0.1)*buffer_size

    # Check if the congestion control is CUBIC or RENO and plot in the corresponding subplot
    if exp['loss_cc'] == 'cubic':
        ax1.plot(experiment_dfs[bbr_key].time, bbr_queue_occupancy, label=f"BBR")
        ax1.plot(experiment_dfs[loss_key].time, loss_queue_occupancy, label=f"CUBIC")
        ax1.set_title("BBR vs CUBIC")
        ax1.set_xlabel("Time (s)")
        ax1.set_ylabel("Queue Occupancy (Packets)")
        ax1.legend(loc="upper center", ncol=2)
    elif exp['loss_cc'] == 'reno':
        ax2.plot(experiment_dfs[bbr_key].time, bbr_queue_occupancy, label=f"BBR")
        ax2.plot(experiment_dfs[loss_key].time, loss_queue_occupancy, label=f"RENO")
        ax2.set_title("BBR vs RENO")
        ax2.set_xlabel("Time (s)")
        ax2.set_ylabel("Queue Occupancy (Packets)")
        ax2.legend(loc="upper center", ncol=2)

plt.tight_layout()
plt.show()

In [ ]:
(bdp_kbyte*exp['bufcap']*1000)/1500

### Reproduce Figure 5

In [ ]:
router_node = slice.get_node("router")
sender_node = slice.get_node("sender")
receiver_node = slice.get_node("receiver")

In [ ]:
import itertools
exp_factors = { 
    'bufcap': [32],
    'duration': [600],
    'loss_cc': ['cubic'],
    'bbrcap':[2],
    'trial': [1]
}
factor_names = [k for k in exp_factors]
factor_lists = list(itertools.product(*exp_factors.values()))
exp_lists = [dict(zip(factor_names, factor_l)) for factor_l in factor_lists]

In [ ]:
import time

# make sure BBR is available
sender_node.execute("sudo modprobe tcp_bbr")
receiver_node.execute("sudo modprobe tcp_bbr")

for exp in exp_lists:
    print("Running:",exp)
    sender_node.execute("sudo modprobe tcp_bbr")
    # set router buffer limit 
    router_node.execute("sudo tc qdisc replace dev " + router_egress_name + " parent 1:3 bfifo limit " + str(bdp_kbyte*exp['bufcap']) + "kb")

    # clean up
    receiver_node.execute("sudo killall iperf3")
    sender_node.execute("sudo killall iperf3")
    
    time.sleep(5) 
    
    # start an iperf3 receiver for the BBR flow
    receiver_node.execute_thread("iperf3 -s -1 -i 1 -fm --logfile fig5_bbr_" + str(exp['bufcap']) +"_" +str(exp['bbrcap'])+ "_bbrV" + exp['loss_cc'] + ".txt")
    # start an iperf3 receiver for the loss based CC flows (Cubic & Reno)
    receiver_node.execute_thread("iperf3 -s -1 -i 1 -fm --logfile fig5_" + exp['loss_cc'] + "_" + str(exp['bufcap']) +"_" + str(exp['bbrcap']) + "_bbrV" + exp['loss_cc'] + ".txt -p 5301")

    time.sleep(10) 

    # start an iperf3 sender for the BBR flow
    sender_node.execute_thread("iperf3 -c receiver -fm -t " + str(exp['duration']) + " -C bbr ")
    # start an iperf3 sender for the loss based CC flows (Cubic & Reno)
    sender_node.execute_thread("iperf3 -c receiver -fm -t " + str(exp['duration']) + " -C " + exp['loss_cc'] + " -p 5301")

    time.sleep(exp['duration'] + 10)
    
print("done")

In [ ]:
experiment_dfs = {}

for exp in exp_lists:

    bbr_file = "fig5_bbr_" + str(exp['bufcap']) +"_" + str(exp['bbrcap']) + "_bbrV" + exp['loss_cc'] + ".txt"
    tput_bbr =  receiver_node.execute("head --lines=-5 /home/ubuntu/figure5/"+bbr_file+" | grep 'Mbits/sec' | awk -F '-' '{print $2}' | awk '{print $1\",\"$5}'", quiet=True)
    df_bbr = pd.read_csv(StringIO(tput_bbr[0]), names = ['time','goodput'])
    experiment_key = f"bbr_{exp['bufcap']}_{exp['loss_cc']}"
    experiment_dfs[experiment_key] = df_bbr


    loss_file = "fig5_" + exp['loss_cc'] + "_" + str(exp['bufcap'])  +"_" + str(exp['bbrcap']) + "_bbrV" + exp['loss_cc'] + ".txt"
    tput_loss = receiver_node.execute("head --lines=-5 /home/ubuntu/figure5/"+loss_file+" | grep 'Mbits/sec' | awk -F '-' '{print $2}' | awk '{print $1\",\"$5}'", quiet=True)
    df_loss = pd.read_csv(StringIO(tput_loss[0]), names = ['time','goodput'])
    experiment_key = f"loss_{exp['bufcap']}_{exp['loss_cc']}"
    experiment_dfs[experiment_key] = df_loss

In [ ]:
from ware_model import *
dict_model=ware_BBR_Model(40, 10, 32, 1, 600)
bbr_model_occupancy=dict_model['simple_model_bbr_occupancy']

In [ ]:
#fig, (ax1, ax2) = plt.subplots(1, 1, figsize=(5, 5))  

fig, ax1 =  plt.subplots(1, 1, figsize=(9, 5)) 

for exp in exp_lists:
    bbr_key = f"bbr_{exp['bufcap']}_{exp['loss_cc']}"
    loss_key = f"loss_{exp['bufcap']}_{exp['loss_cc']}"
    
    buffer_size=(bdp_kbyte*exp['bufcap']*1000)/1500
    
    
    
    bbr_queue_occupancy = (experiment_dfs[bbr_key].goodput * 0.1)*buffer_size
    loss_queue_occupancy = (experiment_dfs[loss_key].goodput * 0.1)*buffer_size

    # Check if the congestion control is CUBIC or RENO and plot in the corresponding subplot
    if exp['bbrcap'] == 2:
        ax1.plot(experiment_dfs[bbr_key].time, bbr_queue_occupancy, label=f"BBR - 2BDP Cap")
        ax1.plot(experiment_dfs[loss_key].time, loss_queue_occupancy, label=f"CUBIC")
        ax1.set_title("BBR vs CUBIC")
        ax1.set_xlabel("Time (s)")
        ax1.set_ylabel("Queue Occupancy (Packets)")
        ax1.axhline(y=bbr_model_occupancy, color='black', linestyle='--', label='Model')
        ax1.legend(loc="upper center", ncol=2)
    elif exp['bbrcap'] == 4:
        ax2.plot(experiment_dfs[bbr_key].time, bbr_queue_occupancy, label=f"BBR - 4BDP Cap")
        ax2.plot(experiment_dfs[loss_key].time, loss_queue_occupancy, label=f"CUBIC")
        ax2.set_title("BBR vs RENO")
        ax2.set_xlabel("Time (s)")
        ax2.set_ylabel("Queue Occupancy (Packets)")
        ax2.legend(loc="upper center", ncol=2)

plt.tight_layout()
plt.savefig("figure5a.pdf")
plt.show()

### Reproduce Figure 6a - 6b

In [ ]:
exp = {'rtt': 40, 'bandwidth': 15 }
bdp_kbyte = exp['rtt']*exp['bandwidth']/8

router_node = slice.get_node("router")
router_ingress_iface = router_node.get_interface(network_name = "net0")
router_ingress_name = router_ingress_iface.get_device_name()
router_egress_iface  = router_node.get_interface(network_name = "net1")
router_egress_name = router_egress_iface.get_device_name()

router_node.execute("sudo tc qdisc del dev " + router_ingress_name + " root")
router_node.execute("sudo tc qdisc del dev " + router_egress_name + " root")

# set up RTT
router_node.execute("sudo tc qdisc replace dev " + router_ingress_name + " root netem delay " + str(exp['rtt']) + "ms limit 10000")
# set up rate limit, buffer limit
router_node.execute("sudo tc qdisc replace dev " + router_egress_name + " root handle 1: htb default 3")
router_node.execute("sudo tc class add dev " + router_egress_name + " parent 1: classid 1:3 htb rate " + str(exp['bandwidth']) + "Mbit")
router_node.execute("sudo tc qdisc add dev " + router_egress_name + " parent 1:3 bfifo limit " + str(bdp_kbyte*32) + "kb")

In [ ]:
# check base delay
_ = slice.get_node("sender").execute("ping -c 5 receiver")

In [ ]:
router_node = slice.get_node("router")
sender_node = slice.get_node("sender")
receiver_node = slice.get_node("receiver")

In [ ]:
import itertools
exp_factors = { 
    'bufcap': [64],
    'duration': [600],
    'bbr_flow_number':[2**(n+1) for n in range(6)],
    'trial': [1]
}
factor_names = [k for k in exp_factors]
factor_lists = list(itertools.product(*exp_factors.values()))
exp_lists = [dict(zip(factor_names, factor_l)) for factor_l in factor_lists]

In [ ]:
exp_lists

In [ ]:
import time

# make sure BBR is available
sender_node.execute("sudo modprobe tcp_bbr")
receiver_node.execute("sudo modprobe tcp_bbr")

for exp in exp_lists:
    print("Running:",exp)
    sender_node.execute("sudo modprobe tcp_bbr")
    # set router buffer limit 
    router_node.execute("sudo tc qdisc replace dev " + router_egress_name + " parent 1:3 bfifo limit " + str(bdp_kbyte*exp['bufcap']) + "kb")

    # clean up
    receiver_node.execute("sudo killall iperf3")
    sender_node.execute("sudo killall iperf3")
    
    time.sleep(5) 
    
    # start an iperf3 receiver for the BBR flow
    receiver_node.execute_thread("iperf3 -s -1 -i 1 -fm --logfile fig6a_bbr_" + str(exp['bufcap']) + "_"+ str(exp['bbr_flow_number']) + ".txt")
    bbr_script="rm -f {name}-ss.txt; start_time=$(date +%s); while true; do ss --no-header -eipn dst 10.0.1.100 | ts '%.s' | tee -a {name}-ss.txt; current_time=$(date +%s); elapsed_time=$((current_time - start_time));  if [ $elapsed_time -ge {duration} ]; then break; fi; sleep 0.1; done;"

    time.sleep(10) 

    # start an iperf3 sender for the BBR flow
    sender_node.execute_thread(bbr_script.format(name="fig6a_bbr_" + str(exp['bufcap']) + "_"+ str(exp['bbr_flow_number']), duration=exp['duration']))
    time.sleep(1) 
    sender_node.execute_thread("iperf3 -c receiver -fm -t " + str(exp['duration']) + " -C bbr -P "+str(exp['bbr_flow_number']))    
    time.sleep(exp['duration'] + 10)
    
print("done")

In [ ]:
def generate_script(name): 
    return f"""
        rm -f {name}-ss.csv
        cat {name}-ss.txt | sed -e ":a; /<->$/ {{ N; s/<->\\n//; ba; }}"  | grep "iperf3" | grep -v "SYN-SENT" | grep -v "cubic" > {name}-ss-processed.txt 
        cat {name}-ss-processed.txt | awk '{{print $1}}' > ts-{name}.txt 
        cat {name}-ss-processed.txt | grep -oP 'bw:\K\d+bps' | sed 's/bps//' > bw-{name}.txt 
        cat {name}-ss-processed.txt | grep -oP '\\bcwnd:.*?(\s|$)' | awk -F '[:,]' '{{print $2}}' | tr -d ' ' > cwnd-{name}.txt
        cat {name}-ss-processed.txt | grep -oP 'mrtt:\K\d+\.\d+' | tr -d ' ' > mrtt-{name}.txt 
        cat {name}-ss-processed.txt | grep -oP '\\bfd=.*?(\s|$)' | awk -F '[=,]' '{{print $2}}' | tr -d ')' | tr -d ' '   > fd-{name}.txt
        paste ts-{name}.txt fd-{name}.txt cwnd-{name}.txt bw-{name}.txt mrtt-{name}.txt -d ',' > {name}-ss.csv
    """

In [ ]:
for exp in exp_lists:
    
    name_bbr="fig6a_bbr_" + str(exp['bufcap']) + "_"+ str(exp['bbr_flow_number'])
    
    script_bbr = generate_script(name_bbr)
    
    sender_node.execute(script_bbr)

In [ ]:
sender_node.upload_file("/home/fabric/work/BBR-Replication/analysis_with_ss.py", f"/home/ubuntu/analysis.py")
sender_node.execute(f'python3 /home/ubuntu/analysis.py')

In [ ]:
sender_node.download_file("/home/fabric/work/BBR-Replication/est_bw.json",f"/home/ubuntu/est_bw.json")
sender_node.download_file("/home/fabric/work/BBR-Replication/est_rtt.json",f"/home/ubuntu/est_rtt_data.json")

In [ ]:
import json
import os

# Initialize empty variables
est_bw = {}
est_rtt = {}

# Directory containing JSON files
data_directory = '/home/fabric/work/BBR-Replication/'

# List of JSON files in the directory
json_files = [f for f in os.listdir(data_directory) if f.endswith('.json')]

# Load data from each JSON file and update the variables
for file_name in json_files:
    file_path = os.path.join(data_directory, file_name)
    with open(file_path, 'r') as f:
        data = json.load(f)

    # Check if the file contains throughput data or srtt data based on its name
    if 'bw' in file_name:
        est_bw.update(data)
    elif 'rtt' in file_name:
        est_rtt.update(data)

In [ ]:
from ware_model import *

model_rtt_est={}
model_inflight_est={}

for exp in exp_lists:
    dict_model=ware_BBR_Model(40, 15, exp['bufcap'], exp['bbr_flow_number'], exp['duration'])
    name="fig6a_bbr_" + str(exp['bufcap']) + "_"+ str(exp['bbr_flow_number'])
    model_inflight_est[name]=dict_model['inflight_cap']
    model_rtt_est[name]=dict_model['RTT_est_ms']

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

N=exp_factors['bbr_flow_number']

model_RTT_est = [model_rtt_est['fig6a_bbr_64_' + str(n)] for n in N]
model_inflight = [model_inflight_est['fig6a_bbr_64_' + str(n)] for n in N]


exp_RTT_est = [est_rtt['fig6a_bbr_64_' + str(n)] for n in N]
exp_inflight = [2*est_bw['fig6a_bbr_64_' + str(n)]*est_rtt['fig6a_bbr_64_' + str(n)]/(8*1000000) for n in N]



# Create the figure with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 6))

# Plot for the first subplot (RTT estimation)
ax1.plot(N, model_RTT_est, color='orange', linestyle='-', label='Model')
ax1.plot(N, exp_RTT_est, color='blue', linestyle='-', label='Actual')
ax1.scatter(N, model_RTT_est, color='orange', s=50)
ax1.scatter(N, exp_RTT_est, color='blue', s=50)
ax1.set_xscale('log', base=2)
ax1.set_xticks(N)
ax1.set_xticklabels(N)
ax1.set_title('RTT Estimation over Number of BBR Flows (Log Scale)')
ax1.set_xlabel('Number of BBR flows')
ax1.set_ylabel('RTT_est (ms)')
ax1.set_ylim(0, max(np.max(model_RTT_est), np.max(exp_RTT_est)) + 5)
ax1.legend()

# Plot for the second subplot (Inflight estimation)
ax2.plot(N, model_inflight, color='orange', linestyle='-', label='Model')
ax2.plot(N, exp_inflight, color='blue', linestyle='-', label='Actual')
ax2.scatter(N, model_inflight, color='orange', s=50)
ax2.scatter(N, exp_inflight, color='blue', s=50)
ax2.set_xscale('log', base=2)
ax2.set_xticks(N)
ax2.set_xticklabels(N)
ax2.set_title('Inflight Estimation over Number of BBR Flows (Log Scale)')
ax2.set_xlabel('Number of BBR flows')
ax2.set_ylabel('Inflight_est (Kbytes)')
ax2.set_ylim(0, max(model_inflight) + 5)
ax2.legend()

# Adjust layout and display the plot
plt.tight_layout()
plt.savefig("figure6.pdf")
plt.show()

### Reproduce Figure 7 - Model 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from ware_model import *

# queue capacity as multiple of BDP
X =  2**np.arange(0, 7)  # 1 to 64

dict_model=ware_BBR_Model(40, 10, X, 1, 400)


plt.figure(figsize=(10, 6))
plt.plot(X, dict_model["probe_time"], color='gray', linestyle='dashed', label='Expected Probe Time')

# Set the x-axis to a logarithmic scale with base 2
plt.xscale('log', base=2)

# Create the ticks for the x-axis manually
plt.xticks(X, X)

# Add dots at powers of 2
plt.scatter(X, dict_model["probe_time"], color='gray', s=20)

# Add title and axis labels
plt.title('Probe Time over Queue Size (Log Scale)')
plt.xlabel('Queue Size (BDP)')
plt.ylabel('Probe Time (s)')

# Set y-axis to show full range from 0 to RTT_est
plt.ylim(0, np.max(dict_model["probe_time"])+5)

# Add legend
plt.legend()

# Show the plot
plt.show()

### Reproduce Figure 8 

In [ ]:
router_node = slice.get_node("router")
sender_node = slice.get_node("sender")
receiver_node = slice.get_node("receiver")

In [ ]:
router_ingress_iface = router_node.get_interface(network_name = "net0")
router_ingress_name = router_ingress_iface.get_device_name()
router_egress_iface  = router_node.get_interface(network_name = "net1")
router_egress_name = router_egress_iface.get_device_name()

In [ ]:
import itertools

# Base factors that remain constant across specific combinations
exp_factors = {
    'bufcap': [2**n for n in range(7)],
    'duration': [1000],
    'bbr_flow_number':[2**(n) for n in range(6)],
    'trial': [1]
}

# List the specific combinations you need
specific_combinations = [
    {'rtt': 40, 'bandwidth': 10, 'loss_cc': 'cubic'},
#    {'rtt': 40, 'bandwidth': 10, 'loss_cc': 'reno'},
#    {'rtt': 30, 'bandwidth': 50, 'loss_cc': 'cubic'}
]

# Generate all possible combinations of the base factors
base_combinations = list(itertools.product(*[[(k, v) for v in values] for k, values in exp_factors.items()]))

# Convert list of tuples to list of dictionaries
base_combinations = [dict(items) for items in base_combinations]

# Expand the specific combinations with the base combinations
exp_lists = [
    {**base, **specific}
    for base in base_combinations
    for specific in specific_combinations
]

In [ ]:
import time

# make sure BBR is available
sender_node.execute("sudo modprobe tcp_bbr")
receiver_node.execute("sudo modprobe tcp_bbr")

for exp in exp_lists:
    print("Running:",exp)
    sender_node.execute("sudo modprobe tcp_bbr")
    
    router_node.execute("sudo tc qdisc del dev " + router_ingress_name + " root")
    router_node.execute("sudo tc qdisc del dev " + router_egress_name + " root")
    router_node.execute("sudo tc qdisc replace dev " + router_ingress_name + " root netem delay " + str(exp['rtt']) + "ms limit 10000")
    router_node.execute("sudo tc qdisc replace dev " + router_egress_name + " root handle 1: htb default 3")
    router_node.execute("sudo tc class replace dev " + router_egress_name + " parent 1: classid 1:3 htb rate " + str(exp['bandwidth']) + "Mbit")
    
    # set router buffer limit 
    bdp_kbyte = exp['rtt']*exp['bandwidth']/8
    router_node.execute("sudo tc qdisc replace dev " + router_egress_name + " parent 1:3 bfifo limit " + str(bdp_kbyte*exp['bufcap']) + "kb")
    
    # clean up
    receiver_node.execute("sudo killall iperf3")
    sender_node.execute("sudo killall iperf3")
    
    time.sleep(5) 
    
    # start an iperf3 receiver for the BBR flow
    receiver_node.execute_thread("iperf3 -s -1 -i 1 -fm --logfile fig8a_bbr_" + str(exp['bufcap'])+"_"+str(exp['bandwidth'])+"_"+str(exp['rtt']) + "_bbrV" + exp['loss_cc'] + "_"+ str(exp['bbr_flow_number'])+ ".txt")
    
    # start an iperf3 receiver for the loss based CC flows (Cubic & Reno)
    receiver_node.execute_thread("iperf3 -s -1 -i 1 -fm --logfile fig8a_" + exp['loss_cc'] + "_" + str(exp['bufcap']) +"_"+str(exp['bandwidth'])+"_"+str(exp['rtt'])+ "_bbrV" + exp['loss_cc'] + "_"+ str(exp['bbr_flow_number'])+ ".txt -p 5301")

    time.sleep(10) 

    # start an iperf3 sender for the BBR flow
    sender_node.execute_thread("iperf3 -c receiver -fm -t " + str(exp['duration']) + " -C bbr -P "+str(exp['bbr_flow_number']))
    # start an iperf3 sender for the loss based CC flows (Cubic & Reno)
    sender_node.execute_thread("iperf3 -c receiver -fm -t " + str(exp['duration']) + " -C " + exp['loss_cc'] + " -p 5301")

    time.sleep(exp['duration'] + 10)
    
print("done")

In [ ]:
experiment_dfs = {}
convergence_time=400
for exp in exp_lists:
    
    bbr_file = "fig8a_bbr_" + str(exp['bufcap']) +"_"+str(exp['bandwidth'])+"_"+str(exp['rtt'])+ "_bbrV" + exp['loss_cc']+"_"+ str(exp['bbr_flow_number']) + ".txt"
    loss_file = "fig8a_" + exp['loss_cc']+"_"+str(exp['bufcap']) +"_"+str(exp['bandwidth'])+"_"+str(exp['rtt'])+ "_bbrV" + exp['loss_cc']+"_"+ str(exp['bbr_flow_number']) + ".txt"

    if exp['bbr_flow_number']==1:
        tput_bbr =  receiver_node.execute("tail -n " +str(convergence_time+4)+ " /home/ubuntu/figure8a/"+bbr_file+" | head -n "+str(convergence_time)+" | grep 'Mbits/sec' | awk -F '-' '{print $2}' | awk '{print $1\",\"$5}'", quiet=True)
    else:
        tput_bbr =  receiver_node.execute("tail -n " +str((exp['bbr_flow_number'] + 2) * convergence_time + 2 * exp['bbr_flow_number'] + 4)+ " /home/ubuntu/figure8a/"+bbr_file+" | head -n "+str((exp['bbr_flow_number'] + 2) * convergence_time + -1)+" | grep 'SUM' | awk -F '-' '{print $2}' | awk '{print $1\",\"$5}'", quiet=True)
    #print(tput_bbr)
    tput_loss =  receiver_node.execute("tail -n " +str(convergence_time+4)+ " /home/ubuntu/figure8a/"+loss_file+" | head -n "+str(convergence_time)+" | grep 'Mbits/sec' | awk -F '-' '{print $2}' | awk '{print $1\",\"$5}'", quiet=True)
    
    
    df_bbr = pd.read_csv(StringIO(tput_bbr[0]), names = ['time','goodput'])
    df_loss = pd.read_csv(StringIO(tput_loss[0]), names = ['time','goodput'])
    average_goodput = df_bbr['goodput'].mean()
    average_goodput_loss=df_loss['goodput'].mean()
    experiment_key = f"bbr_{exp['bufcap']}_{exp['loss_cc']}_{exp['bbr_flow_number']}_{exp['bandwidth']}_{exp['rtt']}"
    experiment_key_loss=f"{exp['loss_cc']}_{exp['bufcap']}_{exp['loss_cc']}_{exp['bbr_flow_number']}_{exp['bandwidth']}_{exp['rtt']}"
    experiment_dfs[experiment_key] = average_goodput  
    experiment_dfs[experiment_key_loss] = average_goodput_loss
                                   
        

# Assuming you want to see the results:
print(experiment_dfs)

In [ ]:
print(len(experiment_dfs))

In [ ]:
plt.figure(figsize=(10, 10))

for i, NBBR in enumerate(exp_factors['bbr_flow_number'], 1):
    dict_model = ware_BBR_Model(40, 10, np.array(exp_factors['bufcap']), NBBR, 400)
    BBR_frac_model = dict_model['BBR_fraction']

    # Plot model data
    plt.subplot(3, 2, i)
    plt.plot(exp_factors['bufcap'], BBR_frac_model, 's-', color='orange', label='Model')

    # Prepare to plot experimental data
    exp_data = []
    for key, value in experiment_dfs.items():
        parts = key.split('_')
        if int(parts[3]) == NBBR:  # Matching the BBR flow number
            if parts[0] == "bbr":
                queue_size = int(parts[1]) 
                exp_data.append((queue_size, value/10))
    
    exp_data.sort()
    exp_queue_sizes, exp_values = zip(*exp_data) if exp_data else ([], [])

    # Plot experimental data
    plt.plot(exp_queue_sizes, exp_values, 'o-', color='blue', label='Actual')
    
    # Subplot formatting
    plt.title(f"{NBBR} BBR Flow{'s' if NBBR > 1 else ''}")
    plt.xscale('log', base=2)
    plt.xticks(exp_factors['bufcap'], exp_factors['bufcap'])
    plt.xlabel('Queue Size (BDP)')
    plt.ylabel('BBR Flows Aggregate Link Fraction')
    plt.legend()

# Adjust overall layout
plt.suptitle('40ms × 10 Mbps, vs 1 Cubic Flow', y=0.99)
plt.tight_layout()
plt.savefig("figure8a.pdf")
plt.show()

In [ ]:
plt.figure(figsize=(10, 10))

for i, NBBR in enumerate(exp_factors['bbr_flow_number'], 1):
    dict_model = ware_BBR_Model(40, 10, np.array(exp_factors['bufcap']), NBBR, 400)
    BBR_frac_model = dict_model['BBR_fraction']

    plt.subplot(3, 2, i)
    plt.plot(exp_factors['bufcap'], BBR_frac_model, 's-', color='orange', label='Model')

    bbr_data = {}
    cubic_data = {}
    for key, value in experiment_dfs.items():
        parts = key.split('_')
        if parts[0] == "bbr" or parts[0] == "cubic":
            flow_type = parts[0]
            queue_size = int(parts[1]) 
            bbr_flow_number = int(parts[3])  # BBR flow number is now clearly identified

            if bbr_flow_number == NBBR:
                if flow_type == "bbr":
                    bbr_data[queue_size] = value
                elif flow_type == "cubic":
                    cubic_data[queue_size] = value

    # Compute the ratio of BBR to Cubic goodput and sort by queue size
    ratios = [(qs, bbr_data[qs] / (bbr_data[qs]+cubic_data[qs])) for qs in bbr_data if qs in cubic_data]
    ratios.sort()
    exp_queue_sizes, ratio_values = zip(*ratios) if ratios else ([], [])

    plt.plot(exp_queue_sizes, ratio_values, 'o-', color='blue', label='Actual')

    plt.title(f"{NBBR} BBR Flow{'s' if NBBR > 1 else ''}")
    plt.xscale('log', base=2)
    plt.xticks(exp_factors['bufcap'], exp_factors['bufcap'])
    plt.xlabel('Queue Size (BDP)')
    plt.ylabel('Actual')
    plt.legend()

plt.suptitle('40ms × 10 Mbps, vs 1 Cubic Flow', y=0.99)
plt.tight_layout()
plt.savefig("figure8a.pdf")
plt.show()

### Delete your slice

When you finish your experiment, you should delete your slice! The following cells deletes all the resources in your slice, freeing them for other experimenters.

In [ ]:
slice = fablib.get_slice(name=slice_name)
fablib.delete_slice(slice_name)

In [ ]:
# slice should end up in "Dead" state
# re-run this cell until you see it in "Dead" state
slice.update()
_ = slice.show()